## Installation of Packages
Restart session after installation to avoid numpy/scipy import errors

In [ ]:
!pip install --prefer-binary pyscf
!pip install openfermion==1.7
!pip install openfermionpyscf
!pip install opt_einsum

# If you have access to GPU, some functions make use of python package CuPy. However, it is recommended to avoid this as it was only experimental.
# !pip install cupy

## Clone the Github repository

If you want to save it in google drive for future use and make updates, first mount the drive and change the directory to a folder inside drive, then clone the repo.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/your-folder          #Change your-folder with the name of the folder in drive where you want to clone the repo

!git clone https://github.com/Shashank-G-M/Perturbative_Trotter_Error.git

%cd Perturbative_Trotter_Error

## Importing the Modules and Packages


In [57]:
from pathlib import Path
import numpy as np

from typing import Union
import scipy as sp
from scipy.sparse.linalg import expm, eigs, eigsh
from scipy.sparse import csc_matrix, csr_matrix, bsr_matrix
from scipy.linalg import logm
from scipy.optimize import minimize, Bounds
from scipy.optimize import differential_evolution, NonlinearConstraint
from scipy.stats import pearsonr

from itertools import product
from itertools import accumulate
from itertools import permutations, combinations

from functools import reduce
from joblib import Parallel, delayed
from copy import copy
from opt_einsum import contract
import matplotlib.pyplot as plt

import time
import math
import random
import pickle
import h5py


import openfermion as of
from openfermion.hamiltonians import fermi_hubbard
from openfermion.transforms import jordan_wigner, reverse_jordan_wigner
from openfermion.linalg import qubit_operator_sparse
from openfermion.linalg import get_number_preserving_sparse_operator
from openfermion.utils import count_qubits, is_hermitian
from openfermion import normal_ordered, chemist_ordered, commutator
from openfermion import eigenspectrum
from openfermion import get_sparse_operator
from openfermion import (
    FermionOperator,
    QubitOperator,
    MolecularData,
    hermitian_conjugated,
    get_molecular_data,
)
from openfermionpyscf import run_pyscf
from openfermion.transforms import get_fermion_operator
from openfermion import jw_hartree_fock_state
from openfermion import MajoranaOperator
from openfermion.hamiltonians import s_squared_operator, sz_operator, number_operator

from pyscf import gto, scf, lo, fci
from pyscf.cc.addons import spatial2spin

import sys
import os


from pert_trotter.cost_utils import *
from pert_trotter.tensor_utils import *
from pert_trotter.ffrag_utils import *
from pert_trotter.fock_utils import *
from pert_trotter.ham_utils import *
from pert_trotter.io_utils import *
from pert_trotter.taper_utils import *
from pert_trotter.trotter_utils import *
from pert_trotter.fermi_frag import *
from pert_trotter.error_pert import *
from pert_trotter.qubit_utils import get_fc_group
from pert_trotter.qubit_utils import get_qwc_group
from pert_trotter.qubit_utils import get_greedy_grouping
from pert_trotter.qubit_utils import Do_Qubit_Partitioning
from pert_trotter import config

## Build molecular Hamiltonian

All molecules except NH$_3$ are generated on the fly. NH$_3$ Hamiltonian, however, is loaded from the repository. To reproduce the results from the paper, use ```r=1``` for H$_2$, LiH, and BeH$_2$, and ```r=1.9``` for H$_2$O and NH$_3$.

We demonstrate the usage for the case of LiH.

In [58]:
r = 1
mol_name = "lih"  # Can be 'h2', 'beh2', 'h2o', and 'nh3'.
if mol_name == "nh3" and r == 1.9:
    with open("data/mol_data/nh3_mol_data.pkl", "rb") as in_file:
        mol_data = pickle.load(in_file)
    H = mol_data["FermionOperator"]
    num_elecs = mol_data["num_elecs"]
    n_qubits = count_qubits(H)
    H_const, H_obt, H_tbt = get_chem_tensors(H)
    H_obt_op, H_tbt_op = obt2op(H_obt), tbt2op(H_tbt)
    Hchem = H_obt_op + H_tbt_op + H_const
else:
    H, num_elecs = obtain_OF_hamiltonian(mol_name, geometry=r)
    n_qubits = count_qubits(H)
    print(num_elecs, n_qubits)
    H_const, H_obt, H_tbt = get_chem_tensors(H)
    H_obt_op, H_tbt_op = obt2op(H_obt), tbt2op(H_tbt)
    Hchem = H_obt_op + H_tbt_op + H_const

config.mol_name = mol_name
config.n_qubits = n_qubits
config.num_elecs = num_elecs
savepath = config.savepath

JW_OF = jordan_wigner(Hchem)

4 12


## Generate exact and approximate eigenstates in the full space

### For non-tapered Hamiltonians

Use this for smaller molecules i.e. exclude NH$_3$. Create the following directory to save the eigenstates.\
```data/JW_OF/mol_name```\
where ```mol_name``` should be replace by the name of the molecule for which data is being produced.

In [59]:
# Generating the exact eigenstates
JW_OF_Array_sparse = qubit_operator_sparse(JW_OF)
JW_OF_Array = JW_OF_Array_sparse.toarray()
v, w = sp.linalg.eigh(JW_OF_Array)
v0 = v[0]
w0 = w[:, [0]]
w0_sparse = sp.sparse.csc_matrix(w[:, [0]])

# Generating the CISD approximated eigenstates
proj_H, all_excitations = get_CI_proj_ham(
    JW_OF_Array,
    n_excitations=2,
    ret_excitations=True,
    num_elecs=num_elecs,
    n_qubits=n_qubits,
)
all_excitations_vec = np.zeros((2**n_qubits, len(all_excitations)))
for i in range(len(all_excitations)):
    all_excitations_vec[:, [i]] = occ_str_to_state(all_excitations[i])
CISD_evals, CISD_evecs = np.linalg.eigh(proj_H)
CISD_evecs_full_space = all_excitations_vec @ CISD_evecs

# Regordering the CISD states so that the first vector corresponds to S^2 = 0
Ssq = s_squared_operator(n_qubits // 2)
Ssq_full_sparse = get_sparse_operator(Ssq, n_qubits)
for i in range(len(CISD_evals)):
    vec = sp.sparse.csc_matrix(CISD_evecs_full_space[:, [i]])
    overlap = np.abs(vec.T * Ssq_full_sparse * vec)[0, 0]
    if np.round(overlap) == 0:
        print(f"Position of ground state of {mol_name} with S^2 = 0: ", i)
        break
CISD_evecs_full_space[i], CISD_evecs_full_space[0] = (
    CISD_evecs_full_space[0],
    CISD_evecs_full_space[i],
)
CISD_evals[i], CISD_evals[0] = CISD_evals[0], CISD_evals[i]
w0_sparse_CISD = sp.sparse.csc_matrix(CISD_evecs_full_space[:, [0]])

print("My CISD gs energy: ", CISD_evals[0])

gs = CISD_evecs_full_space[:, [0]]

# Saving the states. Ensure appropriate directories are available
jw_of = os.path.join(os.getcwd(), "data", "JW_OF", mol_name)
if not os.path.exists(jw_of):
    os.mkdir(jw_of)
p1 = os.path.join(jw_of, f"{mol_name}_v")
p2 = os.path.join(jw_of, f"{mol_name}_w")
p3 = os.path.join(jw_of, f"{mol_name}_CISD_evals")
p4 = os.path.join(jw_of, f"{mol_name}_CISD_evecs")
p5 = os.path.join(jw_of, f"{mol_name}_CISD_evecs_full_space")

with open(p1, "wb") as out_file:
    pickle.dump(v, out_file)
with open(p2, "wb") as out_file:
    pickle.dump(w, out_file)
with open(p3, "wb") as out_file:
    pickle.dump(CISD_evals, out_file)
with open(p4, "wb") as out_file:
    pickle.dump(CISD_evecs, out_file)
with open(p5, "wb") as out_file:
    pickle.dump(CISD_evecs_full_space, out_file)

Position of ground state of H with S^2 = 0:  0
My CISD gs energy:  -7.784451852611049


### For tapered Hamiltonians

Can be used for larger molecules. To reproduce the results for qubit based fragments of NH$_3$, use this method. Create the following directory to save the eigenstates.\
```data/tapered_JW_OF/mol_name```\
where ```mol_name``` should be replace by the name of the molecule for which data is being produced.

In [ ]:
load_ham = False  # Set it to True when you have already saved the tapered Hamiltonian
save_ham = True  # Set it to True when you want to save the tapered Hamiltonian
GPU = False  # Experimental.


if load_ham == True:
    savepath = "data/"
    with open(
        savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_ham_ndarray", "rb"
    ) as in_file:
        tapered_JW_OF_array = pickle.load(in_file)
    # tapered_JW_OF_sarray = sp.sparse.csc_matrix(tapered_JW_OF_array)
    ref_det = "1" * num_elecs + "0" * (n_qubits - num_elecs)
else:
    ref_det = "1" * num_elecs + "0" * (n_qubits - num_elecs)
    tapered_JW_OF = taper_qubits(
        JW_OF, n_qubits, int(num_elecs / 2), int(num_elecs / 2)
    )
    tapered_JW_OF_sarray = get_sparse_operator(tapered_JW_OF, n_qubits - 2)
    tapered_JW_OF_array = np.real(tapered_JW_OF_sarray).toarray()
    if save_ham == True:
        savepath = "data/"
        with open(
            savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_ham_ndarray",
            "wb",
        ) as out_file:
            pickle.dump(tapered_JW_OF_array, out_file)


if GPU:
    import cupy as cp

    tapered_JW_OF_cparray = cp.asarray(tapered_JW_OF_array)
    tap_v_cp, tap_w_cp = cp.linalg.eigh(tapered_JW_OF_cparray)
    tap_v = cp.asnumpy(tap_v_cp)
    tap_w = cp.asnumpy(tap_w_cp)
else:
    tap_v, tap_w = np.linalg.eigh(tapered_JW_OF_array)

tap_w0_sparse = sp.sparse.csc_matrix(tap_w[:, [0]])


tapered_JW_OF_sarray = sp.sparse.csc_matrix(tapered_JW_OF_array)
tapered_CI_states = get_tapered_CI_states(ref_det, [0, 1, 2], preserve_Sz=False)
CIproj_tap_JW_OF_sarray = tapered_CI_states.T * tapered_JW_OF_sarray * tapered_CI_states

CIproj_tap_v0 = eigsh(
    CIproj_tap_JW_OF_sarray, k=1, which="SA", return_eigenvectors=False
)
tap_v0 = eigsh(tapered_JW_OF_sarray, k=1, which="SA", return_eigenvectors=False)

# print ('Error due to tapering: ', np.abs(sym_v[0]-tap_v0))
# print ('Error due to CISD + tapering: ', np.abs(sym_v[0]-CIproj_tap_v0))
# print ('Error between CISD and tapered CISD: ', np.abs(CIproj_tap_v0 - H_NSz_CISD_v[0]))

tap_CISD_evals, tap_CISD_evecs = np.linalg.eigh(CIproj_tap_JW_OF_sarray.toarray())
tap_CISD_evecs_full_space = tapered_CI_states * tap_CISD_evecs


with open(
    savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_v", "wb"
) as in_file:
    pickle.dump(tap_v, in_file)

dump_ndarray_h5(
    savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_w.h5", tap_w
)


with open(
    savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_CISD_evals", "wb"
) as in_file:
    pickle.dump(tap_CISD_evals, in_file)

with open(
    savepath
    + "tapered_JW_OF/"
    + mol_name
    + "/"
    + mol_name
    + "_tap_CISD_evecs_full_space",
    "wb",
) as in_file:
    pickle.dump(tap_CISD_evecs_full_space, in_file)

#### Saving the eigenstates and eigenvalues

## Building projector: $N = \eta,\ S_z=0,\ $and$\ S^2 = 0$

Unliike qubit fragments, fermionic fragments have the same symmetry as the Hamiltonian. This enables us to perform calculations in the symmetry subspace of the ground state of the Hamiltonian. In this section, we build the projector onto this subspace. We also project CISD approximated eigenstates onto this subspace.

In [53]:
Ssq = s_squared_operator(n_qubits // 2)
Ssq_sparse = get_number_preserving_sparse_operator(
    Ssq, n_qubits, num_elecs, spin_preserving=True
)

Ssq_array = Ssq_sparse.toarray()
Ssq_v, Ssq_w = np.linalg.eigh(Ssq_array)
Ssq_w_sparse = sp.sparse.csc_matrix(Ssq_w)
# Ssq_w0_sparse = sp.sparse.csc_matrix(Ssq_w[:,[0]])

counter = 0
for i in range(len(Ssq_v)):
    if Ssq_v[i] <= 0.01:
        counter += 1
non_cisd_dim = counter
Ssq_evals, NSz2SSq_Proj = Ssq_v[:counter], Ssq_w[:, :counter].T
NSz2SSq_Proj_sparse = sp.sparse.csc_matrix(NSz2SSq_Proj)


def get_projected_sparse_op(
    H_OF: Union[FermionOperator, QubitOperator],
    n_qubits=n_qubits,
    num_elecs=num_elecs,
    spin_preserving=True,
    excitation_level=None,
    reference_determinant=None,
):  # H_OF is a FermionOperator in full space
    if type(H_OF) == QubitOperator:
        H_OF = normal_ordered(reverse_jordan_wigner(H_OF))
    first_projected_op = get_number_preserving_sparse_operator(
        H_OF,
        n_qubits,
        num_elecs,
        spin_preserving=spin_preserving,
        excitation_level=excitation_level,
        reference_determinant=reference_determinant,
    )
    if excitation_level == None:
        return NSz2SSq_Proj_sparse * first_projected_op * NSz2SSq_Proj_sparse.T
    else:
        print("Invalid excitation level. Should be 2 or None.")
        return None


def get_projected_sparse_vec(
    vec, CISD=False
):  # vec must be a M x 1 sparse operator where M is the dimension of N = \eta, Sz = 0 subspace.
    if CISD == False:
        return NSz2SSq_Proj_sparse * vec


H_Proj_sparse = get_projected_sparse_op(
    Hchem, n_qubits, num_elecs, spin_preserving=True
)
H_Proj_Array = H_Proj_sparse.toarray()
# Hamiltonian eigenstates projected onto number of electrons = num_elecs, Sz = 0, and S^2 = 0 subspace
sym_v, sym_w = np.linalg.eigh(H_Proj_Array)
sym_w0_sparse = sp.sparse.csc_matrix(sym_w[:, [0]])

## Generating Hamiltonian Partitioning

We generate 4 kinds of qubit Hamiltonian fragments (QWCSI, QWCLF, FCSI, FCLF) and 5 kinds of fermionic Hamiltonian fragments (LRLCU, GFROLCU, LR, GFRO, SDGFRO). Refer to Appendix A of the paper for more detials.

### Non tapered

Before proceeding, create the following tree of directories.\
```data/method/sparse_arrays/full_space```

For example, to generate QWCSI fragments, following directories should be present.\
```data/qwcsi/sparse_arrays/full_space```

For large systems, uncomment the last line to store the data as a hdf5 file instad of a pickle dump.

In [56]:
save, load, spacial = True, False, True

method = "lr"
method_path = f"data/{method}/sparse_arrays/full_space/"
full_path = os.path.join(os.getcwd(), method_path)
Path(full_path).mkdir(parents=True, exist_ok=True)

if method in ["lr", "gfro", "lrlcu", "gfrolcu", "sdgfro"]:
    fragments_list_sarray = Do_Fermi_Partitioning(
        Hchem,
        type=method,
        tol=1e-6,
        spacial=spacial,
        save=save,
        load=load,
        projector_func=get_projected_sparse_op,
    )
else:
    fragments_list = Do_Qubit_Partitioning(JW_OF, type=method)
    # IMPORTANT!!!!! For Sorted insertion based methods (qwcsi and fcsi), manually add the appropriate identity operator to the first fragment, as they lack the identity.
    if method in ["qwcsi", "fcsi"]:
        fragments_list[0] += JW_OF.constant * QubitOperator.identity()
    with open(
        savepath + method + "/" + mol_name + "_" + method + "_frag_ops.pkl", "wb"
    ) as out_file:
        pickle.dump(fragments_list, out_file)
    fragments_list_sarray = [
        get_sparse_operator(frag, n_qubits) for frag in fragments_list
    ]

for f in fragments_list_sarray:
    print(f)
with open(method_path + mol_name + "_sarray.pkl", "wb") as in_file:
    pickle.dump(fragments_list_sarray, in_file)
# dump_list_of_sparse2_h5(sarray_savepath + 'Full space/' + mol_name + '_sarray.h5', fragments_list_sarray)

  (0, 0)	0.52917721092
  (1, 1)	0.5291772109199999
  (2, 2)	0.52917721092
  (0, 0)	(-3.0448814427824424+0j)
  (1, 1)	(-2.536492390311326+0j)
  (2, 2)	(-2.0281033378402102+0j)
  (0, 0)	(0.018270426713393215+0j)
  (1, 1)	(2.0562820100458036e-06+0j)
  (2, 2)	(0.017503341325404053+0j)
  (0, 0)	(0.19679058348547+0j)
  (2, 0)	(0.19679058348547+0j)
  (1, 1)	(0.3935811669709399+0j)
  (0, 2)	(0.19679058348547+0j)
  (2, 2)	(0.19679058348547+0j)
  (0, 0)	(1.2345345723456427+0j)
  (1, 1)	(1.261441330073749+0j)
  (2, 2)	(1.2886381525597437+0j)


### Tapered
Before proceeding, create the following tree of directories.\
```data/tapered_method/sparse arrays/Full space```

For example, to generate QWCSI fragments, following directories should be present.\
```data/tapered_qwcsi/sparse arrays/Full space```

In [ ]:
tapered_JW_OF = taper_qubits(JW_OF, n_qubits, int(num_elecs / 2), int(num_elecs / 2))
method = "qwcsi"
fragments_list = Do_Qubit_Partitioning(tapered_JW_OF, type=method)
# IMPORTANT!!!!! For Sorted insertion based methods (qwcsi and fcsi), manually add the appropriate identity operator to the first fragment, as they lack the identity.
if method in ["qwcsi", "fcsi"]:
    fragments_list[0] += tapered_JW_OF.constant * QubitOperator.identity()
with open(
    savepath + "tapered_" + method + "/" + mol_name + "_" + method + "_frag_ops.pkl",
    "wb",
) as out_file:
    pickle.dump(fragments_list, out_file)


fragments_list_sarray = [
    get_sparse_operator(frag, n_qubits - 2) for frag in fragments_list
]
sarray_savepath = savepath + "tapered_" + method + "/sparse arrays/"
dump_list_of_sparse2_h5(
    sarray_savepath + "Full space/" + mol_name + "_sarray.h5", fragments_list_sarray
)

## Calculate exact and approximate perturbative Trotter error $(\varepsilon_2,
 \varepsilon_{\text{app}})$

 Assumes you have generated and stored relevant fragments and eigenstates

### For fermionic fragments

In [26]:
try:
    fragments_list_sarray = list(
        load_h5_list_of_sparse(method_path + mol_name + "_sarray.h5")
    )
except FileNotFoundError:
    with open(method_path + mol_name + "_sarray.pkl", "rb") as in_file:
        fragments_list_sarray = list(pickle.load(in_file))

v, w, w0_sparse = sym_v, sym_w, sym_w0_sparse
CISD_v, CISD_w, CISD_w0_sparse = (
    H_NSz_CISD_v,
    H_NSzSsq_CISD_full_space_w,
    H_NSzSsq_CISD_full_space_w0_sparse,
)

# Second Order Trotter
print("Full space (\u03f5_2):")
v2_contri = efficient_v2_contri_2_order_Trotter(fragments_list_sarray, w0_sparse)
print("2nd order Trotter v2 contribution: ", v2_contri)

print("\n\nCISD space (\u03f5_app):")
v2_contri = efficient_v2_contri_2_order_Trotter(fragments_list_sarray, CISD_w0_sparse)
print("2nd order Trotter v2 contribution: ", v2_contri)

Full space (ϵ_2):
Total number of loops:  9
Progress: [50%]

2nd order Trotter v2 contribution:  (0.0027744548423093065+0j)


CISD space (ϵ_app):
Total number of loops:  9
Progress: [50%]

2nd order Trotter v2 contribution:  (0.0027744548423093065+0j)


### For qubit fragments

In [ ]:
method = "qwcsi"
mol_name = "lih"
istapered = False  # Setting this to True assumes you have generated and saved the tapered data for the current molecule


if istapered == True:
    sarray_savepath = savepath + "tapered_" + method + "/sparse arrays/"
    fragments_list_sarray = load_h5_list_of_sparse(
        sarray_savepath + "Full space/" + mol_name + "_sarray.h5"
    )

    with open(
        savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_v", "rb"
    ) as in_file:
        v = pickle.load(in_file)
    w = load_h5_ndarray(
        savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_w.h5"
    )
    w0_sparse = sp.sparse.csc_matrix(w[:, [0]])

    with open(
        savepath + "tapered_JW_OF/" + mol_name + "/" + mol_name + "_tap_CISD_evals",
        "rb",
    ) as in_file:
        CISD_v = pickle.load(in_file)
    with open(
        savepath
        + "tapered_JW_OF/"
        + mol_name
        + "/"
        + mol_name
        + "_tap_CISD_evecs_full_space",
        "rb",
    ) as in_file:
        CISD_w = pickle.load(in_file)
    CISD_w0_sparse = sp.sparse.csc_matrix(CISD_w[:, [0]])
else:
    sarray_savepath = savepath + method + "/sparse arrays/"
    with open(
        sarray_savepath + "Full space/" + mol_name + "_sarray.pkl", "rb"
    ) as in_file:
        fragments_list_sarray = pickle.load(in_file)

    with open(savepath + "JW_OF/" + mol_name + "/" + mol_name + "_v", "rb") as in_file:
        v = pickle.load(in_file)

    try:
        w = load_h5_ndarray(savepath + "JW_OF/" + mol_name + "/" + mol_name + "_w.h5")
    except FileNotFoundError:
        with open(
            savepath + "JW_OF/" + mol_name + "/" + mol_name + "_w", "rb"
        ) as in_file:
            w = pickle.load(in_file)
    w0_sparse = sp.sparse.csc_matrix(w[:, [0]])

    with open(
        savepath + "JW_OF/" + mol_name + "/" + mol_name + "_CISD_evals", "rb"
    ) as in_file:
        CISD_v = pickle.load(in_file)
    with open(
        savepath + "JW_OF/" + mol_name + "/" + mol_name + "_CISD_evecs_full_space", "rb"
    ) as in_file:
        CISD_w = pickle.load(in_file)
    CISD_w0_sparse = sp.sparse.csc_matrix(CISD_w[:, [0]])


# Second order Trotter
print("Full space (\u03f5_2):")
v2_contri = efficient_v2_contri_2_order_Trotter(fragments_list_sarray, w0_sparse)
print("2nd order Trotter v2 contribution: ", v2_contri)

print("\n\nCISD space (\u03f5_app):")
v2_contri = efficient_v2_contri_2_order_Trotter(fragments_list_sarray, CISD_w0_sparse)
print("2nd order Trotter v2 contribution: ", v2_contri)

## Evaluating $\alpha$

Refer to Eq. (4) of the paper for the definition of $\alpha$

In [ ]:
# # import cupy as cp
# # import cupyx.scipy.sparse as cusparse
# # from cupyx.scipy.sparse.linalg import eigsh as gpu_eigsh

method = "lrlcu"
mol_name = "lih"
istapered = False

sarray_savepath = savepath + method + "/sparse arrays/"

if istapered == True:
    try:
        fragments_list_sarray = load_h5_list_of_sparse(
            savepath
            + "tapered_"
            + method
            + "/sparse arrays/Full space/"
            + mol_name
            + "_sarray.h5"
        )
    except FileNotFoundError:
        with open(
            savepath
            + "tapered_"
            + method
            + "/sparse arrays/Full space/"
            + mol_name
            + "_sarray.pkl",
            "rb",
        ) as in_file:
            fragments_list_sarray = pickle
else:
    try:
        fragments_list_sarray = load_h5_list_of_sparse(
            sarray_savepath + "Full space/" + mol_name + "_sarray.h5"
        )
    except FileNotFoundError:
        with open(
            sarray_savepath + "Full space/" + mol_name + "_sarray.pkl", "rb"
        ) as in_file:
            fragments_list_sarray = pickle.load(in_file)

# fragments_list_sarray_gpu = [cusparse.csc_matrix(frag, dtype=cp.complex128) for frag in fragments_list_sarray]

alpha = Trotter_2nd_order_alpha_error(fragments_list_sarray, gpu=False)
print(alpha)

## Working with Trotter Unitary

Get the effective Hamiltonian from U_trotter by applying matrix logarithm to U_trotter unitary. This cell also calculates all other relevant quantities concerning U_trotter and $\hat{H}_{\text{eff}}$ like $\alpha_e$ (see Eq. (10) in paper),$\ \varepsilon,\ E_0,\ E_0^{T},\ \Delta E_T,\ $and time step $t$.

In [ ]:
method = "lrlcu"
mol_name = "lih"
istapered = False

if istapered == True:
    try:
        fragments_list_sarray = load_h5_list_of_sparse(
            savepath
            + "tapered_"
            + method
            + "/sparse arrays/Full space/"
            + mol_name
            + "_sarray.h5"
        )
    except FileNotFoundError:
        with open(
            savepath
            + "tapered_"
            + method
            + "/sparse arrays/Full space/"
            + mol_name
            + "_sarray.pkl",
            "rb",
        ) as in_file:
            fragments_list_sarray = pickle
else:
    try:
        fragments_list_sarray = load_h5_list_of_sparse(
            savepath
            + method
            + "/sparse arrays/"
            + "Full space/"
            + mol_name
            + "_sarray.h5"
        )
    except FileNotFoundError:
        with open(
            savepath
            + method
            + "/sparse arrays/"
            + "Full space/"
            + mol_name
            + "_sarray.pkl",
            "rb",
        ) as in_file:
            fragments_list_sarray = pickle.load(in_file)

print("Check point 1")

frags_len = len(fragments_list_sarray)

Happ = sp.sparse.csc_matrix(sum(fragments_list_sarray))
v0 = sp.sparse.linalg.eigsh(Happ, k=1, which="SA", tol=1e-8, return_eigenvectors=False)[
    0
]
spec_norm = np.abs(
    sp.sparse.linalg.eigsh(Happ, k=1, which="LM", tol=1e-8, return_eigenvectors=False)[
        0
    ]
)
dt = 1 / spec_norm

print("Check point 2")


U_trotter = sp.sparse.eye(Happ.shape[0], format="csc")
counter = 1
for frag in fragments_list_sarray:
    print("scipy exponentiation initiated (via sp.sparse.linalg.expm)")
    U_trotter_frag = sp.sparse.linalg.expm(
        -1.0j * frag * dt / 2
    )  # dt/2 because we are eventually going to construct second order Trotter unitary
    print("scipy exponentiation complete")

    U_trotter = U_trotter * U_trotter_frag

    print(f"Loop : {counter}/{frags_len} \n")
    counter += 1
U_trotter_array = U_trotter.toarray()

print("Check point 3")


U_trotter_array_2nd_order = U_trotter_array @ U_trotter_array.T
H_eff_array_2nd_order = 1.0j * sp.linalg.logm(U_trotter_array_2nd_order) / (dt)
H_eff_sarray_2nd_order = sp.sparse.csc_matrix(H_eff_array_2nd_order)

print("Check point 4")

v0_eff = sp.sparse.linalg.eigsh(
    H_eff_sarray_2nd_order, k=1, which="SA", tol=1e-8, return_eigenvectors=False
)[0]

DE_T = np.abs(v0_eff - v0)
epsilon = DE_T / dt**2

print("Epsilon = ", epsilon)

U_trotter_sarray_2nd_order = sp.sparse.csc_matrix(U_trotter_array_2nd_order)

U_exact_sarray = sp.sparse.linalg.expm(-1.0j * Happ * dt)
alpha_e_op = U_exact_sarray - U_trotter_sarray_2nd_order
alpha_e = np.abs(
    sp.sparse.linalg.eigs(alpha_e_op, k=1, which="LM", return_eigenvectors=False)[0]
)
alpha_e = alpha_e / dt**3

print("alpha_e = ", alpha_e)

print("dt = ", dt)

H_eff_info = {
    "mol_name": mol_name,
    "fragmentation": method,
    "E0": v0,
    "E0_eff": v0_eff,
    "DeltaE_T": DE_T,
    "epsilon": epsilon,
    "alpha_e": alpha_e,
    "time_step": dt,
}

print(H_eff_info)

with open(
    savepath
    + method
    + "/sparse arrays/"
    + "Full space/"
    + mol_name
    + "_H_eff_info.pkl",
    "wb",
) as out_file:
    pickle.dump(H_eff_info, out_file)

## Correlation Plots

Producing figure 1 from paper using tables provided in the paper. All the values in the tables can be generated by using the cells above.

In [ ]:
import matplotlib

a = 0
b = 5

c = 0
d = 9

matplotlib.rcParams.update({"font.size": 13})

eps_v2_exact = np.array(
    [
        [
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0027744548,
            0.002780725,
            0.00300493,
        ],
        [
            0.0032480968,
            0.002163644,
            0.0030127983,
            0.00244501,
            0.003300397,
            0.003386966,
            0.04718898,
            0.049932963,
            0.01818811,
        ],
        [
            0.013731045,
            0.011020856,
            0.02272475,
            0.008758768,
            0.009295491,
            0.009634088,
            0.028728955,
            0.03341760,
            0.0195634463,
        ],
        [
            0.006219993,
            0.004758676,
            0.02356259,
            0.00284857,
            0.023670257,
            0.02508310,
            0.14204710,
            0.128333,
            0.02582307,
        ],
        [
            0.01148842,
            0.00997528,
            0.089486699,
            0.0152268,
            0.019967049,
            0.0198038678,
            0.165155677,
            0.137869267,
            0.029367855,
        ],
    ]
)
eps_v2_exact = eps_v2_exact[a:b, c:d]


eps_v2_app = np.array(
    [
        [
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0027744548,
            0.002780725,
            0.00300493,
        ],
        [
            0.0032552085,
            0.00217699,
            0.003022389,
            0.002458995,
            0.003304652,
            0.003386966,
            0.047186947,
            0.049938408,
            0.01818605,
        ],
        [
            0.01382715,
            0.011163517,
            0.02250201,
            0.008934338,
            0.00949379,
            0.009828665,
            0.0289486374,
            0.03363103,
            0.0197738062,
        ],
        [
            0.009778753,
            0.00804437,
            0.019877499,
            0.006027446,
            0.032246767,
            0.035204712,
            0.177783456,
            0.16698050,
            0.03484607,
        ],
        [
            0.01311337,
            0.01565665,
            0.07786943,
            0.01048145,
            0.033328745,
            0.0344475977,
            0.23392218,
            0.20878358,
            0.0474685,
        ],
    ]
)
eps_v2_app = eps_v2_app[a:b, c:d]

eps = np.array(
    [
        [
            0.0033079,
            0.003308,
            0.00330791,
            0.0033079,
            0.0033079,
            0.0033079,
            0.0028361,
            0.00284251,
            0.0030695,
        ],
        [
            0.0032497,
            0.002168,
            0.00301,
            0.002449,
            0.00330,
            0.00339,
            0.047050,
            0.049808,
            0.018174,
        ],
        [
            0.0137506,
            0.011032,
            0.022746,
            0.008771,
            0.009307,
            0.009646,
            0.02869,
            0.03334,
            0.01956,
        ],
        [
            0.006223,
            0.004761,
            0.023572,
            0.002850,
            0.023673,
            0.025086,
            0.142037,
            0.128308,
            0.025828,
        ],
        [
            0.011495,
            0.009978,
            0.08958,
            0.015229,
            0.019971,
            0.019808,
            0.165136,
            0.1378337,
            0.0293757,
        ],
    ]
)
eps = eps[a:b, c:d]


# Creating subplots
fig, axes = plt.subplots(1, 2, figsize=(11, 4))


y = eps.flatten()
y = y / max(y)
x1 = eps_v2_exact.flatten()
x1 = x1 / max(x1)
x2 = eps_v2_app.flatten()
x2 = x2 / max(x2)


# Obtaining correlation coefficients
r1, _ = pearsonr(list(x1), list(y))
r2, _ = pearsonr(list(x2), list(y))


axes[1].scatter(
    x1,
    y,
    alpha=0.7,
    marker=".",
    c="r",
    label="$\epsilon_{\\text{2}}$ correlation = %.2f" % r1,
)
axes[1].scatter(
    x2,
    y,
    alpha=0.7,
    marker="*",
    c="b",
    label="$\epsilon_{\\text{app}}$ correlation = %.2f" % r2,
)

axes[1].set_xlim(0, 1.1)
axes[1].set_ylim(0, 1.1)

# Fit and plot a trendline (optional)
m1, b1 = np.polyfit(x1, y, 1)
axes[1].plot(x1, m1 * x1 + b1, color="red", alpha=0.3, linestyle="-")

m2, b2 = np.polyfit(x2, y, 1)
axes[1].plot(x2, m2 * x2 + b2, color="blue", linestyle="-")

axes[1].set_xlabel("Perturbation based Trotter error", fontsize=14)
# axes[1].set_ylabel("Exact Trotter error ($\epsilon$)", fontsize=14)
axes[1].legend()


# Time steps
dt = np.array([[0.908141], [0.128461], [0.064592], [0.01337], [0.018123]])


# alpha
alpha = np.array(
    [
        [
            0.019917,
            0.016297,
            0.02223,
            0.016297,
            0.016296,
            0.016297,
            0.015327,
            0.015319,
            0.015776,
        ],
        [
            1.071179,
            0.261750,
            0.62713,
            0.260248,
            0.130109,
            0.117276,
            0.523921,
            0.461217,
            0.230157,
        ],
        [
            4.22008,
            1.023566,
            4.96021,
            0.986088,
            0.580984,
            0.550564,
            2.359864,
            2.03442,
            1.12533,
        ],
        [
            79.40951,
            28.73135,
            181.5595,
            27.85735,
            15.30047,
            15.05653,
            52.37091,
            48.27083,
            27.88276,
        ],
        [
            51.66221,
            16.36468,
            65.99235,
            16.0206,
            7.81380,
            7.640403,
            28.428811,
            25.78507,
            14.50743,
        ],
    ]
)
alpha = alpha[a:b, c:d]
alpha_dt2 = alpha


alpha_e = np.array(
    [
        [
            0.018603,
            0.011459,
            0.01860,
            0.011459,
            0.011459,
            0.011459,
            0.01111,
            0.01111,
            0.011299,
        ],
        [
            0.22472,
            0.180268,
            0.2506,
            0.180367,
            0.10034,
            0.09980,
            0.07223,
            0.08325,
            0.06039,
        ],
        [
            0.672117,
            0.75365,
            0.80521,
            0.756595,
            0.42353,
            0.42957,
            0.29231,
            0.34624,
            0.35711,
        ],
        [
            23.25445,
            23.42057,
            46.44921,
            23.44921,
            1.84682,
            11.86137,
            15.66702,
            14.87779,
            12.86540,
        ],
        [
            11.22894,
            11.22535,
            15.86476,
            11.21336,
            6.552417,
            6.559554,
            7.254640,
            7.048216,
            6.025332,
        ],
    ]
)
alpha_e = alpha_e[a:b, c:d]
alpha_e_dt2 = alpha_e


yy = eps.flatten()
yy = yy / max(yy)
xx1 = alpha_dt2.flatten()
xx1 = xx1 / max(xx1)
xx2 = alpha_e_dt2.flatten()
xx2 = xx2 / max(xx2)


rr1, _ = pearsonr(list(xx1), list(yy))
rr2, _ = pearsonr(list(xx2), list(yy))


axes[0].scatter(
    xx1, yy, alpha=0.7, marker=".", c="r", label="$\\alpha$ correlation = %.2f" % rr1
)
axes[0].scatter(
    xx2, yy, alpha=0.7, marker="*", c="b", label="$\\alpha_e$ correlation = %.2f" % rr2
)

axes[0].set_xlim(0, 1.1)
axes[0].set_ylim(0, 1.1)

# Fit and plot a trendline (optional)
m1, b1 = np.polyfit(xx1, yy, 1)
axes[0].plot(xx1, m1 * xx1 + b1, color="red", alpha=0.7, linestyle="-")

m2, b2 = np.polyfit(xx2, yy, 1)
axes[0].plot(xx2, m2 * xx2 + b2, color="blue", linestyle="-")

axes[0].set_xlabel("Operator norm based Trotter error", fontsize=14)
axes[0].set_ylabel("Exact Trotter error ($\epsilon$)", fontsize=14)
axes[0].legend()


axes[1].text(
    -0.01,
    1.02,
    "a)",
    transform=axes[1].transAxes,
    fontsize=14,
    fontweight="bold",
    va="top",
    ha="right",
)
axes[0].text(
    -0.01,
    1.02,
    "b)",
    transform=axes[0].transAxes,
    fontsize=14,
    fontweight="bold",
    va="top",
    ha="right",
)


fig.tight_layout()

# #To save the figure, uncomment the following if working on google colab
# from google.colab import files
# plt.savefig('Trotter_error_correlation_plot.pdf')
# files.download('Trotter_error_correlation_plot.pdf')
plt.show()

## T gate counts
Calculate upperbound on T gates needed to reach chemical accuracy in obtaining ground state energy of molecular Hamiltonians under QPE + Trotter approximation algorithm for various Hamiltonian fragmentation techniques.

### Finding the best T gates by sampling initial values for optimizer

Since the function that optimize T gate count is highly sensitive to the inital parameters, we optimize T gates for various initial parameters on a grid to get a better estimate.

In [ ]:
eps_v2_exact = np.array(
    [
        [
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0027744548,
            0.002780725,
            0.00300493,
        ],
        [
            0.0032480968,
            0.002163644,
            0.0030127983,
            0.00244501,
            0.003300397,
            0.003386966,
            0.04718898,
            0.049932963,
            0.01818811,
        ],
        [
            0.013731045,
            0.011020856,
            0.02272475,
            0.008758768,
            0.009295491,
            0.009634088,
            0.028728955,
            0.03341760,
            0.0195634463,
        ],
        [
            0.006219993,
            0.004758676,
            0.02356259,
            0.00284857,
            0.023670257,
            0.02508310,
            0.14204710,
            0.128333,
            0.02582307,
        ],
        [
            0.01148842,
            0.00997528,
            0.089486699,
            0.0152268,
            0.019967049,
            0.0198038678,
            0.165155677,
            0.137869267,
            0.029367855,
        ],
    ]
)


eps_v2_app = np.array(
    [
        [
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0032412139,
            0.0027744548,
            0.002780725,
            0.00300493,
        ],
        [
            0.0032552085,
            0.00217699,
            0.003022389,
            0.002458995,
            0.003304652,
            0.003386966,
            0.047186947,
            0.049938408,
            0.01818605,
        ],
        [
            0.01382715,
            0.011163517,
            0.02250201,
            0.008934338,
            0.00949379,
            0.009828665,
            0.0289486374,
            0.03363103,
            0.0197738062,
        ],
        [
            0.009778753,
            0.00804437,
            0.019877499,
            0.006027446,
            0.032246767,
            0.035204712,
            0.177783456,
            0.16698050,
            0.03484607,
        ],
        [
            0.01311337,
            0.01565665,
            0.07786943,
            0.01048145,
            0.033328745,
            0.0344475977,
            0.23392218,
            0.20878358,
            0.0474685,
        ],
    ]
)


alpha = np.array(
    [
        [
            0.019917,
            0.016297,
            0.02223,
            0.016297,
            0.016296,
            0.016297,
            0.015327,
            0.015319,
            0.015776,
        ],
        [
            1.071179,
            0.261750,
            0.62713,
            0.260248,
            0.130109,
            0.117276,
            0.523921,
            0.461217,
            0.230157,
        ],
        [
            4.22008,
            1.023566,
            4.96021,
            0.986088,
            0.580984,
            0.550564,
            2.359864,
            2.03442,
            1.12533,
        ],
        [
            79.40951,
            28.73135,
            181.5595,
            27.85735,
            15.30047,
            15.05653,
            52.37091,
            48.27083,
            27.88276,
        ],
        [
            51.66221,
            16.36468,
            65.99235,
            16.0206,
            7.81380,
            7.640403,
            28.428811,
            25.78507,
            14.50743,
        ],
    ]
)


mol_names = ["h2", "lih", "beh2", "h2o", "nh3"]
all_methods = [
    "qwclf",
    "qwcsi",
    "fclf",
    "fcsi",
    "lrlcu",
    "gfrolcu",
    "lr",
    "gfro",
    "sdgfro",
]
rs = [1, 1, 1, 1.9, 1.9]
scale = 1e8
Tot_Err = 1.6e-3
already_tapered = False
n_samps = 40

# Arrays to store T gate estimates
eps_v2_exact_T_count = np.zeros((len(mol_names), len(all_methods)))
eps_v2_app_T_count = np.zeros((len(mol_names), len(all_methods)))
alpha_T_count = np.zeros((len(mol_names), len(all_methods)))

# Finding T gates for all the samples in one run
for i, mol_name in enumerate(mol_names):
    r = rs[i]
    H, num_elecs = obtain_OF_hamiltonian(mol_name, geometry=r)

    H_const, H_obt, H_tbt = get_chem_tensors(H)
    H_obt_op, H_tbt_op = obt2op(H_obt), tbt2op(H_tbt)
    Hchem = H_obt_op + H_tbt_op + H_const

    for j, method in enumerate(all_methods):
        if j < 4:
            if mol_name != "nh3":
                JW_OF = jordan_wigner(Hchem)
                n_qubits = count_qubits(JW_OF)
                already_tapered = False

            # Perform tapering if it is nh3 and method is a qubit method
            if (mol_name == "nh3") and (not already_tapered):
                JW_OF = taper_qubits(
                    JW_OF, n_qubits, int(num_elecs / 2), int(num_elecs / 2)
                )
                n_qubits = count_qubits(JW_OF)
                already_tapered = True

            JW_keys = JW_OF.terms.keys()
            Nr = 2 * len(JW_keys)  # *2 because we are working with 2nd order Trotter
        else:
            try:
                with open(
                    savepath
                    + f"{method}/sparse arrays/Full space/{mol_name}_sarray.pkl",
                    "rb",
                ) as in_file:
                    frags = pickle.load(in_file)
            except FileNotFoundError:
                frags = load_h5_list_of_sparse(
                    savepath + f"{method}/sparse arrays/Full space/{mol_name}_sarray.h5"
                )
            n_qubits = count_qubits(H)
            Nr = countOneRotFerm(n_qubits, len(frags), ord=2)

        print(f"Currently doing {mol_name} with {method}")

        err_est = eps_v2_exact[i, j]
        global_optimal_errs, global_min_Nt = explore_params(
            n_samps, Tot_Err, err_est, Nr, scale, use_constraints=True
        )
        if sum(global_optimal_errs) - Tot_Err > 1e-8:
            raise ValueError("Sum of global_optimal_errs exceeds Tot_Err")
        eps_v2_exact_T_count[i, j] = global_min_Nt

        err_est = eps_v2_app[i, j]
        global_optimal_errs, global_min_Nt = explore_params(
            n_samps, Tot_Err, err_est, Nr, scale, use_constraints=True
        )
        if sum(global_optimal_errs) - Tot_Err > 1e-8:
            raise ValueError("Sum of global_optimal_errs exceeds Tot_Err")
        eps_v2_app_T_count[i, j] = global_min_Nt

        err_est = alpha[i, j]
        global_optimal_errs, global_min_Nt = explore_params(
            n_samps, Tot_Err, err_est, Nr, scale, use_constraints=False
        )
        if sum(global_optimal_errs) - Tot_Err > 1e-8:
            raise ValueError("Sum of global_optimal_errs exceeds Tot_Err")
        alpha_T_count[i, j] = global_min_Nt

        print(f"Completed T gate count for {mol_name} with {method} \n")


np.set_printoptions(linewidth=95)
np.set_printoptions(formatter={"float": "{:.2e}".format})
print(np.matrix(alpha_T_count))
print(np.matrix(eps_v2_exact_T_count))
print(np.matrix(eps_v2_app_T_count))

### Printing and comparing the best methods based on T gate estimates

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(linewidth=95)
np.set_printoptions(formatter={"float": "{:.2e}".format})
# print (np.matrix(alpha_T_count))
# print (np.matrix(eps_v2_exact_T_count))
# print (np.matrix(eps_v2_app_T_count))


methods = [
    "qwc lf",
    "qwc si",
    "fc lf",
    "fc si",
    "lr lcu",
    "gfro lcu",
    "lr",
    "gfro",
    "sd gfro",
]
mol_names = ["h2", "lih", "beh2", "h2o", "nh3"]

alpha_T_count = np.array(
    [
        [
            1.56e07,
            1.40e07,
            1.65e07,
            1.40e07,
            1.39e08,
            1.05e08,
            1.35e08,
            1.01e08,
            2.76e08,
        ],
        [
            6.22e09,
            2.99e09,
            4.71e09,
            2.98e09,
            2.31e10,
            5.10e10,
            4.75e10,
            1.04e11,
            1.66e11,
        ],
        [
            1.34e10,
            6.42e09,
            1.46e10,
            6.30e09,
            9.17e10,
            2.21e11,
            1.89e11,
            4.35e11,
            7.31e11,
        ],
        [
            1.02e11,
            6.02e10,
            1.56e11,
            5.93e10,
            4.63e11,
            1.06e12,
            8.75e11,
            1.94e12,
            2.83e12,
        ],
        [
            8.16e10,
            4.50e10,
            9.27e10,
            4.45e10,
            5.46e11,
            1.60e12,
            1.06e12,
            3.01e12,
            3.89e12,
        ],
    ]
)


eps_v2_exact_T_count = np.array(
    [
        [
            6.22e06,
            6.22e06,
            6.22e06,
            6.22e06,
            6.19e07,
            4.65e07,
            5.70e07,
            4.29e07,
            1.20e08,
        ],
        [
            3.13e08,
            2.53e08,
            3.01e08,
            2.70e08,
            3.52e09,
            8.32e09,
            1.40e10,
            3.36e10,
            4.58e10,
        ],
        [
            7.01e08,
            6.25e08,
            9.11e08,
            5.55e08,
            1.10e10,
            2.79e10,
            1.98e10,
            5.31e10,
            9.21e10,
        ],
        [
            7.72e08,
            6.72e08,
            1.55e09,
            5.14e08,
            1.66e10,
            3.97e10,
            4.21e10,
            9.25e10,
            7.83e10,
        ],
        [
            1.06e09,
            9.88e08,
            3.09e09,
            1.23e09,
            2.55e10,
            7.56e10,
            7.61e10,
            2.06e11,
            1.62e11,
        ],
    ]
)


eps_v2_app_T_count = np.array(
    [
        [
            6.22e06,
            6.22e06,
            6.22e06,
            6.22e06,
            6.19e07,
            4.65e07,
            5.70e07,
            4.29e07,
            1.20e08,
        ],
        [
            3.13e08,
            2.54e08,
            3.01e08,
            2.70e08,
            3.52e09,
            8.32e09,
            1.40e10,
            3.36e10,
            4.58e10,
        ],
        [
            7.04e08,
            6.29e08,
            9.07e08,
            5.60e08,
            1.11e10,
            2.82e10,
            1.99e10,
            5.33e10,
            9.26e10,
        ],
        [
            9.78e08,
            8.83e08,
            1.41e09,
            7.60e08,
            1.95e10,
            4.73e10,
            4.73e10,
            1.06e11,
            9.14e10,
        ],
        [
            1.14e09,
            1.25e09,
            2.88e09,
            1.01e09,
            3.32e10,
            1.01e11,
            9.11e10,
            2.56e11,
            2.07e11,
        ],
    ]
)

best_alpha_T_count = np.zeros((5, 3))
best_eps_v2_exact_T_count = np.zeros((5, 3))
best_eps_v2_app_T_count = np.zeros((5, 3))

n = 3  # Choose best 3 fragmentation
for i in range(5):
    print(f"Molecule = {mol_names[i]}")

    eps_v2_exact_T_count_i = eps_v2_exact_T_count[i, :]
    # Step 1: Get indices
    smallest_indices = np.argpartition(eps_v2_exact_T_count_i, n)[:n]
    # Step 2: Sort those indices by actual value
    smallest_indices = smallest_indices[
        np.argsort(eps_v2_exact_T_count_i[smallest_indices])
    ]
    # Step 3: Get the corresponding smallest values
    smallest_values = eps_v2_exact_T_count_i[smallest_indices]
    best_eps_v2_exact_T_count[i, :] = smallest_values
    print(
        [
            methods[smallest_indices[j]].upper()
            + " $({:.2e}".format(smallest_values[j])[:6]
            + "\times 10^{"
            + "{:.2e}".format(smallest_values[j])[-2:]
            + "})$"
            for j in range(3)
        ]
    )

    alpha_T_count_i = alpha_T_count[i, :]
    # Step 1: Get indices of n smallest values (unsorted)
    smallest_indices = np.argpartition(alpha_T_count_i, n)[:n]
    # Step 2: Sort those indices by actual value
    smallest_indices = smallest_indices[np.argsort(alpha_T_count_i[smallest_indices])]
    # Step 3: Get the corresponding smallest values
    smallest_values = alpha_T_count_i[smallest_indices]
    best_alpha_T_count[i, :] = smallest_values
    print(
        [
            methods[smallest_indices[j]].upper()
            + " $({:.2e}".format(smallest_values[j])[:6]
            + "\times 10^{"
            + "{:.2e}".format(smallest_values[j])[-2:]
            + "})$"
            for j in range(3)
        ]
    )

    eps_v2_app_T_count_i = eps_v2_app_T_count[i, :]
    # Step 1: Get indices
    smallest_indices = np.argpartition(eps_v2_app_T_count_i, n)[:n]
    # Step 2: Sort those indices by actual value
    smallest_indices = smallest_indices[
        np.argsort(eps_v2_app_T_count_i[smallest_indices])
    ]
    # Step 3: Get the corresponding smallest values
    smallest_values = eps_v2_app_T_count_i[smallest_indices]
    best_eps_v2_app_T_count[i, :] = smallest_values
    print(
        [
            methods[smallest_indices[j]].upper()
            + " $({:.2e}".format(smallest_values[j])[:6]
            + "\times 10^{"
            + "{:.2e}".format(smallest_values[j])[-2:]
            + "})$"
            for j in range(3)
        ]
    )
    print("\n\n")

# print (best_alpha_T_count)
# print (best_eps_v2_exact_T_count)
# print (best_eps_v2_app_T_count)